
# Predicting the Price of Used Cars Using Machine Learning Algorithms
<font color = 'Blue'> 
Name: Shin Le, Jeongyeon Kim, Benjamin Horvath, Nico Reategui, Paul Giglio


<a id="0"></a> <br>
<font color = 'Blue'> 
# Table of Contents

1. [About Dataset](#1)
1. [Importing Libraries](#2)
1. [Functions Implementation](#3)
1. [Loading Data](#4)
1. [Data Preprocessing](#5)
   1. [Outlier](#51)
   1. [Dummy Variables](#52)
   1. [Labeling Encoding](#53)
1. [Exploratory Data Analysis](#6)
1. [Data Splitting](#7)
1. [Models Evaluations and Predictions](#8)
   1. [*Full Model* with *Linear Regression*](#81)
   1. [Using Sequential Feature Selection for the *Linear Regression*](#82)
   1. [*Decision Tree*](#83)
   1. [*A Pruned Tree*](#84)
   1. [Ensemble Method: *Random Forest Classifier*](#85)
   1. [Ensemble Method: *Gradient Boosting Classifier*](#86)
   1. [*Support Vector Machine* (SVM)](#87)


<a id="2"></a>
<font color = 'blue'> 
## **2. Importing Libraries**

In [18]:

import numpy as np
import pandas as pd
from matplotlib.pyplot import subplots
%matplotlib inline
from statsmodels.api import OLS
import statsmodels.api as sm

#preprocessing
from sklearn.preprocessing import StandardScaler, LabelEncoder 

#sklearn
import sklearn.model_selection as skm
import sklearn.linear_model as skl
from sklearn.model_selection import train_test_split,cross_val_predict
from sklearn.decomposition import PCA
from sklearn.cross_decomposition import PLSRegression 
from sklearn.metrics import accuracy_score, confusion_matrix, jaccard_score, classification_report,\
    log_loss, mean_squared_error
from sklearn.model_selection import train_test_split,  cross_val_score
from sklearn.linear_model import LinearRegression, LogisticRegression, LogisticRegressionCV

#tree
from sklearn.tree import DecisionTreeClassifier,plot_tree ,export_text
import sklearn.tree as tree 
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from ISLP.bart import BART
#searborn
import seaborn as sns


#Kaggle and loading data
import os
from kaggle.api.kaggle_api_extended import KaggleApi

from ISLP.models import (ModelSpec as MS, summarize , poly)
from functools import partial
from ISLP.models import Stepwise , sklearn_selected , sklearn_selection_path
from l0bnb import fit_path

#SVM
from sklearn.svm import SVC




In [19]:
import pandas as pd

url = "https://drive.google.com/uc?export=download&id=1J0Nkiyx53MtEkme4kO6Z3uljxC-WWunK"


df = pd.read_csv(url)

In [20]:
df

,brand,model,color,registration_date,year,price_in_euro,power_kw,power_ps,transmission_type,fuel_type,fuel_consumption_l_100km,fuel_consumption_g_km,mileage_in_km,offer_description
0,alfa-romeo,Alfa Romeo GTV,red,10/1995,1995,1300,148,201,Manual,Petrol,10.9,260.0,160500.0,2.0 V6 TB
1,alfa-romeo,Alfa Romeo Spider,black,07/1995,1995,4900,110,150,Manual,Petrol,9.5,225.0,189500.0,2.0 16V Twin Spark L
2,alfa-romeo,Alfa Romeo 164,red,11/1996,1996,17950,132,179,Manual,Petrol,7.2,NaN,96127.0,"3.0i Super V6, absoluter Topzustand !"
3,alfa-romeo,Alfa Romeo Spider,red,04/1996,1996,7900,110,150,Manual,Petrol,9.5,225.0,47307.0,2.0 16V Twin Spark
4,alfa-romeo,Alfa Romeo 145,red,12/1996,1996,3500,110,150,Manual,Petrol,8.8,210.0,230000.0,Quadrifoglio
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
224986,volvo,Volvo XC60,silver,05/2023,2023,55400,145,197,Automatic,Diesel,5.6,142.0,5000.0,B4 Autom. Plus Dark Keyless-Start/Klima/LED/BC
224987,volvo,Volvo XC60,silver,03/2023,2023,54500,145,197,Automatic,Diesel,5.6,142.0,5900.0,B4 Autom. Plus Dark Sitzhzg.
224988,volvo,Volvo XC90,white,03/2023,2023,89690,173,235,Automatic,Diesel,7.6,202.0,4900.0,B5 AWD Diesel Ultimate Dark 7-Sitzer Massage F...
224989,volvo,Volvo V60,white,05/2023,2023,61521,145,197,Automatic,Diesel,4.7,125.0,1531.0,"B4 D Plus Dark 145 kW, 5-türig (Diesel)"
